In [2]:
# Cell 1 — Import libraries & load model/tokenizer

import os
import sys
import re
import numpy as np

from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import pickle

# FIXED → Correct path to src folder
sys.path.append(os.path.abspath("../src"))

# Import Member A's function
from data_preparation import prepare_data

MODEL_PATH = "../saved_model/sentiment_model.h5"
TOKENIZER_PATH = "../saved_model/tokenizer.pkl"
MAX_LEN = 120

model = load_model(MODEL_PATH)

with open(TOKENIZER_PATH, "rb") as f:
    tokenizer = pickle.load(f)

print("Model and tokenizer loaded successfully.")


Model and tokenizer loaded successfully.


In [4]:
#clean and predict 
def clean_text(text: str) -> str:
    """
    Normalise input: lowercase, remove non-alphanumeric chars,
    collapse multiple spaces. MUST match training + Flask.
    """
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def predict_sentiment(text: str):
    """
    Predict sentiment for a single input sentence.
    Returns (label, probability).
    """
    cleaned = clean_text(text)
    seq = tokenizer.texts_to_sequences([cleaned])
    seq = pad_sequences(
        seq,
        maxlen=MAX_LEN,
        padding="post",
        truncating="post"
    )
    prob = float(model.predict(seq, verbose=0)[0][0])
    label = "Positive" if prob >= 0.5 else "Negative"
    return label, prob

test_sentences = [
    "I really love this product so much",
    "This is the worst thing I have ever bought",
    "The quality is okay, not great but not terrible",
    "I am very disappointed with this chair",
    "Amazing value for money, highly recommended!"
]

for s in test_sentences:
    label, prob = predict_sentiment(s)
    print(f"Input: {s}")
    print(f" → Prediction: {label} ({prob:.4f})")
    print("-" * 60)


Input: I really love this product so much
 → Prediction: Positive (0.7037)
------------------------------------------------------------
Input: This is the worst thing I have ever bought
 → Prediction: Negative (0.0357)
------------------------------------------------------------
Input: The quality is okay, not great but not terrible
 → Prediction: Negative (0.0613)
------------------------------------------------------------
Input: I am very disappointed with this chair
 → Prediction: Negative (0.0913)
------------------------------------------------------------
Input: Amazing value for money, highly recommended!
 → Prediction: Positive (0.7693)
------------------------------------------------------------
